# Comparison CUTonalá and PVlib forecast using NDFD

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pvlib
from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

In [2]:
temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

In [3]:
cec_modules = pvlib.pvsystem.retrieve_sam('CECMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

cec_module = cec_modules['Canadian_Solar_Inc__CS6X_320P']
cec_inverter = cec_inverters['INGETEAM_POWER_TECHNOLOGY_S_A___Ingecon_Sun_40TL_U_M__480V_']

In [4]:
latitude= 20.56
longitude= -103.22
altitude=1544
tz='America/Mexico_City'

In [5]:
location = Location(latitude=latitude, longitude=longitude, altitude=altitude, tz=tz)

In [6]:
surface_tilt = 25
surface_azimuth = 180 # pvlib uses 0=North, 90=East, 180=South, 270=West convention
albedo = 0.2

In [7]:
system = PVSystem(surface_tilt=surface_tilt, surface_azimuth=surface_azimuth,
                 module_parameters=cec_module,
                 inverter_parameters=cec_inverter,
                 temperature_model_parameters=temperature_model_parameters,
                 albedo=albedo, surface_type=None, module=None, 
                 module_type='glass_polymer',
                 modules_per_string=19.5, 
                 strings_per_inverter=8, inverter=None, 
                 racking_model='open_rack', 
                 losses_parameters=None, 
                 name='Name of PV system: Huerto de Cutonalá')

In [8]:
mc = ModelChain(system, location, name='Huerto de Cutonalá', 
                clearsky_model='ineichen', transposition_model='haydavies', 
                solar_position_method='nrel_numpy', airmass_model='kastenyoung1989',
                dc_model=None, #from module specifications
                ac_model='sandia', #from inverter specifications
                aoi_model='no_loss', 
                spectral_model='no_loss', #spectral model 'no loss' otherwise error
                temperature_model='sapm', losses_model='no_loss')

In [9]:
import datetime
#Variables de tiempo

start = pd.Timestamp(datetime.date.today(), tz='America/Mexico_City') # today's date
end = start + pd.Timedelta(days=7) # 7 days from today

#9 de diciebre a partir del 24 de diciembre
start9_dec = start - pd.Timedelta(days=15)
end9_dec = start9_dec + pd.Timedelta(days=1)

In [11]:
from pvlib import solarposition, irradiance, atmosphere, pvsystem, inverter, temperature, iam
from pvlib.forecast import NDFD

/Users/luisvargas/Desktop/Tesis Pvlib/env_pvlib_tesis/lib/python3.8/site-packages/pvlib/forecast.py:18: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  warnings.warn(


In [12]:
fm = NDFD()

In [13]:
# Se obtiene la data y se procesa del modelo de pronostico
forecast_data = fm.get_processed_data(latitude, longitude, start9_dec, end9_dec)

HTTPError: Error accessing https://thredds.ucar.edu/thredds/ncss/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/Best?var=Total_cloud_cover_surface&var=Wind_speed_height_above_ground&var=Temperature_height_above_ground&time_start=2020-12-09T00%3A00%3A00-06%3A00&time_end=2020-12-10T00%3A00%3A00-06%3A00&longitude=-103.22&latitude=20.56&accept=netcdf
Server Error (400: Requested Lat/Lon Point (+20.56N 103.2W) is not contained in the Data. Data Bounding Box =  lat= [20.17,50.11] lon= [-130.12,-60.86])

The National Digital Forecast Database (NDFD) is a suite of products generated by the National Weather Service (NWS) using data from regional NWS Weather Forecast Offices (WFOs) and the National Centers for Environmental Prediction (NCEP). Forecast products locally generated at WFOs are digitally entered into the NDFD. Additionally, forecasters at WFOs enhance gridded model data generated by NCEP and enter these data into the NDFD. The products in the NDFD are used by the NWS to make text, graphic, and digital products that are then made available to the public and emergency managers. The NDFD also provides data for commercial weather companies to generate their own products based off of NWS forecasts.